In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from sklearn import tree
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

* preparing the dataset and exploring feuture s'type 

In [ ]:
data = pd.read_csv('../input/solar-power-generation/BigML_Dataset_5f50a4cc0d052e40e6000034.csv')
data.info()

In [ ]:
data.head()

In [ ]:
data.describe().transpose()

**cleaning the data set **

In [ ]:
data = data.dropna(axis=0)

**Exploratory data analysis**

In [ ]:
subdata = data.groupby('Year').sum()['Power Generated']
subdata=subdata.to_frame()
plt.figure(figsize=(16,6))
subdata.plot(kind='pie',legend = True, subplots=True)

the power has producted in 2009 is too biger than taht producted in  2008 
with ration 302.44 % 

In [ ]:
plt.figure(figsize=(16,6))
subdata=data[["Average Temperature (Day)","Power Generated"]].groupby("Average Temperature (Day)").mean()
subdata.head(5).plot(kind="barh",grid=True,title = "the 5 five avg temerature has a the small power generated ",stacked=True,color=['b', 'r', 'g', 'y', 'k'])
subdata.tail(5).plot(kind="barh",grid=True,title = "the 5 five avg temerature has a the big power generated ",stacked=True)

In [ ]:
plt.figure(figsize=(16,6))
subdata.plot()

as we can see the Pv cells work as well as temperature increase but coming to 78 degre the power producted will decrese 


In [ ]:
plt.figure(figsize=(16,6))
subdata = data.groupby("Relative Humidity")["Power Generated"].sum()
subdata.tail(5).plot(kind="barh",grid=True,title = "the 5 five Relative Humidity value has a the big power generated ",stacked=True)
subdata.head(5).plot(kind="barh",grid=True,title = "the 5 five Relative Humidity value has a the small power generated ",stacked=True)

In [ ]:
plt.figure(figsize=(16,6))
subdata.plot()

power generated depend of the relative humidity 

In [ ]:
plt.figure(figsize=(16,6))
subdata = data.groupby("Average Wind Speed (Period)")["Power Generated"].sum()
subdata.tail(5).plot(kind="barh",grid=True,title = "the 5 five wind speed value has a the big power generated ",stacked=True)
subdata.head(5).plot(kind="barh",grid=True,title = "the 5 five wind speed value has a the small power generated ",stacked=True)

In [ ]:
plt.figure(figsize=(16,6))
subdata.plot()

more the wind is slow more the power generated is more big 

**now we want to summrize the probleme and see how the power generated change depend eacch feature**

In [ ]:
data.columns

In [ ]:
sub_data = data[['Is Daylight', 'Distance to Solar Noon', 'Average Temperature (Day)', 'Average Wind Direction (Day)', 'Average Wind Speed (Day)', 'Sky Cover','Visibility', 'Relative Humidity', 'Average Wind Speed (Period)','Average Barometric Pressure (Period)', 'Power Generated']]
plt.figure(figsize=(16,6))
g = sns.PairGrid(sub_data)
g.map(sns.scatterplot)
g.add_legend()

In [ ]:
data.corr()

In [ ]:
f, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(data.corr().abs(), annot=True, linewidths=.5, ax=ax)

In [ ]:
data.columns

In [ ]:
features=['Day of Year', 'Year', 'Month', 'Day', 'First Hour of Period', 'Is Daylight', 'Distance to Solar Noon', 'Average Temperature (Day)', 'Average Wind Direction (Day)', 'Average Wind Speed (Day)', 'Sky Cover','Visibility', 'Relative Humidity', 'Average Wind Speed (Period)','Average Barometric Pressure (Period)']
X = data[features]
y = data["Power Generated"]
clf = tree.DecisionTreeRegressor(max_depth=5)
clf.fit(X,y)
print(clf.score(X,y))
import graphviz 
dot_data = tree.export_graphviz(clf,feature_names = features,class_names="Power Generated",filled=True, rounded=True,special_characters=True, out_file=None) 
graph = graphviz.Source(dot_data) 
graph

in this part i will try apply knn method  for predict power generated based on the 12 feauture

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state=10) # 70% training and 30% test
score_valeus=[]
for i in range(1,100):
    model = KNeighborsRegressor(n_neighbors= i)
    model.fit(X_train,y_train)
    score = model.score(X_test,y_test)
    score_valeus.append(score)
score_valeus
ax = np.arange(len(score_valeus))
plt.figure(figsize=(16,6))
plt.grid()
plt.plot(ax,score_valeus)

the best number of neighbors is 2 neighbors with accurency  of 55% 
as we can see the model is not very good at this moment

In [ ]:
kf = KFold(n_splits=5,shuffle=True, random_state=30 )
X = data[features].values
y = data["Power Generated"].values
scores=[]
for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = KNeighborsRegressor(n_neighbors= 2)
        model.fit(X_train,y_train)
        score = model.score(X_test,y_test)
        scores.append(score)
scores
plt.rcdefaults()
fig, ax = plt.subplots()

people = ('first split', 'secod split', 'third split', 'fourth_splith', 'fivth_split')
y_pos = np.arange(len(people))
performance = scores

ax.barh(y_pos, performance, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(people)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('score')
ax.set_title('score by split')
plt.show()   

best score is 57% honnestly very bad performance of this methode because the size of data is very little 

In [ ]:
kf = KFold(n_splits=5,shuffle=True, random_state=30 )
X = data[features].values
y = data["Power Generated"].values
scores_knn=[]
score_lineare=[]
for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = KNeighborsRegressor(n_neighbors= 2)
        model.fit(X_train,y_train)
        score = model.score(X_test,y_test)
        scores_knn.append(score)
        model=LinearRegression()
        model.fit(X_train,y_train)
        score = model.score(X_test,y_test)
        score_lineare.append(score)
score_lineare
labels = ['first split', 'secod split', 'third split', 'fourth_splith', 'fivth_split']
men_means = [20, 34, 30, 35, 27]
women_means = [25, 32, 34, 20, 25]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, score_lineare, width, label='lineare_models')
rects2 = ax.bar(x + width/2, scores_knn, width, label='knn_models')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Scores by models and split')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()


very well accurency in the lineare model comprated to the knn models with accurency of 69.5%

but removing year month day from list s'feauture acurency of each model will increase

In [ ]:
features=['First Hour of Period', 'Is Daylight', 'Distance to Solar Noon', 'Average Temperature (Day)', 'Average Wind Direction (Day)', 'Average Wind Speed (Day)', 'Sky Cover','Visibility', 'Relative Humidity', 'Average Wind Speed (Period)','Average Barometric Pressure (Period)']
kf = KFold(n_splits=5,shuffle=True, random_state=30 )
X = data[features].values
y = data["Power Generated"].values
scores_knn=[]
score_lineare=[]
for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = KNeighborsRegressor(n_neighbors= 2)
        model.fit(X_train,y_train)
        score = model.score(X_test,y_test)
        scores_knn.append(score)
        model=LinearRegression()
        model.fit(X_train,y_train)
        score = model.score(X_test,y_test)
        score_lineare.append(score)
score_lineare
labels = ['first split', 'secod split', 'third split', 'fourth_splith', 'fivth_split']
men_means = [20, 34, 30, 35, 27]
women_means = [25, 32, 34, 20, 25]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, score_lineare, width, label='lineare_models')
rects2 = ax.bar(x + width/2, scores_knn, width, label='knn_models')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Scores by models and split')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

the new  accurency of knn is 69% and for the lineare model is 73.98%

1.svm models

In [ ]:
features=[ 'First Hour of Period', 'Is Daylight', 'Distance to Solar Noon', 'Average Temperature (Day)', 'Average Wind Direction (Day)', 'Average Wind Speed (Day)', 'Sky Cover','Visibility', 'Relative Humidity', 'Average Wind Speed (Period)','Average Barometric Pressure (Period)']
X = data[features].values
y = data["Power Generated"].values
regr = svm.SVR()
regr.fit(X, y)
ypred=regr.predict(X)
p=mean_absolute_error(ypred,y)
print(p)

2. xboost models

In [ ]:
features=['Day of Year', 'Year', 'Month', 'Day', 'First Hour of Period', 'Is Daylight', 'Distance to Solar Noon', 'Average Temperature (Day)', 'Average Wind Direction (Day)', 'Average Wind Speed (Day)', 'Sky Cover','Visibility', 'Relative Humidity', 'Average Wind Speed (Period)','Average Barometric Pressure (Period)']
X = data[features].values
y = data["Power Generated"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state=10) # 70% training and 30% test
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)
ypred = my_model.predict(X_test)
p=mean_absolute_error(ypred,y_test)
print(p)